In [1]:
import pandas as pd
import numpy as np

In [2]:
# 讀入檔案
df = pd.read_csv('clean_contextV1.csv', index_col=0)
# print(df.shape)
# df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4616 entries, 0 to 4615
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      4616 non-null   object
 1   title       4616 non-null   object
 2   article     4616 non-null   object
 3   hotel_name  2721 non-null   object
 4   keywords    4616 non-null   object
dtypes: object(5)
memory usage: 216.4+ KB


In [3]:
df = df.drop_duplicates(subset='title', keep='first', inplace=False, ignore_index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4425 entries, 0 to 4567
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   author      4425 non-null   object
 1   title       4425 non-null   object
 2   article     4425 non-null   object
 3   hotel_name  2648 non-null   object
 4   keywords    4425 non-null   object
dtypes: object(5)
memory usage: 207.4+ KB


In [4]:
# 檢查是否有空值(non)
check_for_nan = df.isnull().values.any()
print (check_for_nan)
df = df.dropna(subset=['hotel_name'])

True


In [5]:
# df['keywords'] = df['keywords'].apply(lambda x :' '.join(x))
# df['hotel_name']=df['hotel_name'].apply(lambda x:x.replace("'","").replace("[","").replace("]",""))

df['hotel_name']=df['hotel_name'].apply(lambda x:eval(x)) # string轉換為list
df['keywords']=df['keywords'].apply(lambda x:eval(x))
df.head(3)

,author,title,article,hotel_name,keywords
0,mandywei,心得|宜蘭|蘇澳|煙波大飯店蘇澳四季雙泉館,作者mandywei|喂喂不在家|看板Hotel標題|心得|蘇澳|煙波大飯店蘇澳四季雙泉館時...,[煙波大飯店],"[房間, 部分, 飲料, 早餐, 不錯, 房型, 人數, 煙波]"
1,terry910134,心得|宜蘭|麗翔酒店礁溪館|南洋風戶外溫泉池,作者terry910134|懶人愛吃喝|看板Hotel標題|心得|宜蘭|麗翔酒店礁溪館|南洋...,"[麗翔, 麗翔酒店]","[大廳, 泳池, 飯店, 鐵道, 客房, 南洋, 泡湯, 設施]"
2,faigo3ppp,心得|宜蘭|礁溪山泉大飯店住宿含早餐與宵夜,作者faigo3ppp|Dr|Lai|看板Hotel標題|心得|宜蘭|礁溪山泉大飯店住宿含早...,"[晶泉丰旅, 山泉大飯店, 綠舞國際觀光飯店, 中天溫泉, 宜蘭力麗威斯汀度假酒店, 棲蘭山...","[早餐, 宵夜, 飯店, 酒店, 頂樓, 火車, 小孩, 景觀]"


In [6]:
df = df[['author', 'hotel_name', 'keywords']]

# 將hotel_name多的拆分成不同的row
def split_row(data, column):
    row_len = list(map(len, data[column].values))
    rows = []
    for i in data.columns:
        if i == column:
            row = np.concatenate(data[i].values)
        else:
            row = np.repeat(data[i].values, row_len)
        rows.append(row)
    # return row_len
    return pd.DataFrame(np.dstack(tuple(rows))[0], columns=data.columns)

df = split_row(df, column='hotel_name')

In [7]:
# map user id and movie id to integer starting from 0 to N (num of users) and M (num of movies)
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
hotel_encoder = LabelEncoder()

df['uid_int'] = user_encoder.fit_transform(df.author)
df['hotel_int'] = hotel_encoder.fit_transform(df.hotel_name)

文字轉向量 \
tf-idf 效果很不好

In [8]:
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
# warnings.filterwarnings('ignore')

# with open('gensim.txt', 'w', encoding='utf-8') as f:
# df['keywords']=df['keywords'].apply(lambda x:eval(x))
datas = [i for i in df['keywords']]
    # datas = '\n'.join(datas)
    # f.write(datas)
# tf = TfidfVectorizer(max_df = 0.5) # max_df找單詞在文件中的最高出現率，取值範圍為 [0.0~1.0]
# features = tf.fit_transform(datas).toarray()
# print(features)
# df1 = pd.DataFrame(features, columns=['key_features'])


In [9]:
!pip install gensim

In [74]:
from gensim.models import Word2Vec

model = Word2Vec(datas, vector_size=100,min_count=1,workers=7, window =3, sg = 1) #size=50

In [75]:
# # 統一維度
def feature(row):
    vec = np.zeros(100)
    for word in row:
        vec += model.wv[word]
    return (np.average(vec))*100

df['key_feature'] = df['keywords'].apply(feature)

In [12]:
# from sklearn.decomposition import PCA
# X = df['key_feature'][4]
# print((np.average(X)))
# # X_pca = PCA(n_components=2).fit_transform(X)  # whiten=True 表示資料需要正規化
# # print(X_pca)

In [76]:
user_dict = {}
for i, value in enumerate(df['author']):
    user_dict[value] = df['uid_int'][i]
print(max(user_dict.values()))

1313


In [77]:
df['uid_int_small']=df['uid_int'].apply(lambda x: float(x/1313))

In [78]:
df.tail(10)

,author,hotel_name,keywords,uid_int,hotel_int,key_feature,uid_int_small,users_features
5136,Roc4031,棲蘭山莊,"[明體, 中心, 太平山, 建築, 車站, 親水, 公園, 傳藝]",115,408,11.328036,0.087586,"(115, [115.0, 1.1097821457355166])"
5137,Roc4031,太平山莊,"[明體, 中心, 太平山, 建築, 車站, 親水, 公園, 傳藝]",115,186,11.328036,0.087586,"(115, [115.0, 1.1097821457355166])"
5138,Roc4031,幾度,"[明體, 中心, 太平山, 建築, 車站, 親水, 公園, 傳藝]",115,273,11.328036,0.087586,"(115, [115.0, 1.1097821457355166])"
5139,qooqootin,童玩,"[民宿, 夜景, 烏石港, 食材, 布袋, 戲館, 有名, 烤肉]",941,562,9.519897,0.716679,"(941, [941.0, 0.914536189824139])"
5140,qooqootin,途中,"[民宿, 夜景, 烏石港, 食材, 布袋, 戲館, 有名, 烤肉]",941,700,9.519897,0.716679,"(941, [941.0, 0.914536189824139])"
5141,maximum621,微笑橘子民宿,"[民宿, 橘子, 咖啡, 蘇澳港, 中心, 蜜餞, 羅東, 夜市]",766,279,9.528765,0.583397,"(766, [766.0, 0.968416365901794])"
5142,ailhsiao,童玩,"[旅館, 時間, 山河, 路線, 星光, 大鎮, 床頭, 房間]",179,562,9.915859,0.136329,"(179, [179.0, 0.9996022176055703])"
5143,yating0801,近水,"[很棒, 民宿, 熱心, 好險, 房間, 火車, 排隊, 行李]",1242,699,10.554739,0.945925,"(1242, [1242.0, 1.0274532698997063])"
5144,presotea,中冠礁溪大飯店,"[民宿, 味道, 時機, 有名, 最旺, 玩樂, 景點, 逗趣]",932,17,7.464323,0.709825,"(932, [932.0, 0.7302163060783641])"
5145,skyc,太平山莊,"[太平山, 時間, 植物, 風景, 電視, 山路, 山谷, 房間]",1048,186,9.553617,0.798172,"(1048, [1048.0, 0.9425993458076847])"


設置用戶特徵

In [2]:
def create_features(dataframe, features_name, id_col_name):
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features1 = list(zip(dataframe[id_col_name], features))
    features2 = features.apply(pd.Series).stack().reset_index(drop=True)
    return features1, features2

In [3]:
# 用户特征
users_f = ['uid_int','key_feature']

df['users_features'], user_feature_list = create_features(df, users_f, 'uid_int')
print(user_feature_list)
# df.head(1)

NameError: name 'df' is not defined

In [81]:
!pip install lightfm

In [82]:
from sklearn.metrics.pairwise import cosine_similarity
from lightfm import LightFM, cross_validation
from scipy.sparse import csr_matrix, coo_matrix
from lightfm.evaluation import auc_score 
from lightfm.data import Dataset
import numpy as np

In [83]:
# 开始制作数据集
make_dataset = Dataset()
make_dataset.fit(df['uid_int'], df['hotel_int'], user_features=user_feature_list)

In [84]:
# 构建打分矩阵
interactions, weights = make_dataset.build_interactions(
    list(zip(df['uid_int'], df['hotel_int'], df['key_feature'])))

# 打印用户评分
df['key_feature'].unique()

array([13.14716962, 12.53185683, 13.60134239, ..., 10.55473923,
        7.46432326,  9.5536169 ])

數據及劃分

In [85]:
# model load
model_test = LightFM(no_components=30, random_state=1)

# 固定划分数据
train, test = cross_validation.random_train_test_split(
    interactions, test_percentage=0.3)

模型训练

In [86]:
# 用户特征处理
users_features = make_dataset.build_user_features(df['users_features'])

# model train
%time model_test.fit(train, user_features=users_features, epochs=20, verbose=False)

CPU times: user 1.83 s, sys: 0 ns, total: 1.83 s
Wall time: 1.83 s


模型验证

In [87]:
# 开始验证模型
auc = auc_score(model_test, test, user_features=users_features)

# 计算 auc 的均值
np.mean(auc)



0.7397442

In [88]:
# 推荐模型训练
model = LightFM(no_components=30, random_state=1)

%time model.fit(interactions, sample_weight=weights, user_features=users_features, epochs=30,  verbose=False)

CPU times: user 3.07 s, sys: 6.26 ms, total: 3.07 s
Wall time: 3.07 s


In [133]:
# 按照用户名进行推荐
Username = df['author'][163]

# id 转换
user_x = int(user_dict[Username])

# 对电影进行去重
# allready_knews = df.loc[train_df['uid_int'].isin([user_x])].drop_duplicates('hotel_int', keep='first', inplace=False)
allready_knews = df.loc[df['uid_int'].isin([user_x])]

In [134]:
# 过滤看过的内容
known_items = allready_knews['hotel_int'].tolist()
df_use_for_prediction = df.loc[~df['hotel_int'].isin(known_items)].drop_duplicates(
    'hotel_int', keep='first', inplace=False)

# 挑选高质量的电影
# Threshold = 0.6
# df_use_for_prediction = df_use_for_prediction[df_use_for_prediction['key_feature'] > Threshold]
# df.loc[df['hotel_int'].isin(known_items)]

df_use_for_prediction.head(10)



,author,hotel_name,keywords,uid_int,hotel_int,key_feature,uid_int_small,users_features
2,terry910134,麗翔酒店,"[大廳, 泳池, 飯店, 鐵道, 客房, 南洋, 泡湯, 設施]",1120,789,12.531857,0.853008,"(1120, [1120.0, 12.531856834044447])"
21,sailly,和風時尚會館,"[房間, 飯店, 浴缸, 網路, 空間, 家庭, 備品, 宜蘭]",986,130,13.864929,0.750952,"(986, [986.0, 13.86492861044826])"
22,sailly,頭城時尚會館,"[房間, 飯店, 浴缸, 網路, 空間, 家庭, 備品, 宜蘭]",986,756,13.864929,0.750952,"(986, [986.0, 13.86492861044826])"
25,Furella,礁溪寒沐酒店,"[日式, 冷盤區, 不錯, 海鮮, 酒店, 繁多, 品項, 台式]",42,532,9.590538,0.031988,"(42, [42.0, 9.590537532923918])"
26,Furella,寒沐酒店,"[日式, 冷盤區, 不錯, 海鮮, 酒店, 繁多, 品項, 台式]",42,224,9.590538,0.031988,"(42, [42.0, 9.590537532923918])"
27,Furella,礁溪老爺,"[日式, 冷盤區, 不錯, 海鮮, 酒店, 繁多, 品項, 台式]",42,535,9.590538,0.031988,"(42, [42.0, 9.590537532923918])"
30,Furella,眺望,"[房型, 酒店, 時間, 浴池, 舒服, 側則, 印尼, 泰國]",42,528,10.899437,0.031988,"(42, [42.0, 10.899437486368697])"
34,gourmetism,阡陌五房,"[民宿, 阡陌, 房間, 清水, 五房, 距離, 房型, 冰鎮]",496,734,9.279634,0.377761,"(496, [496.0, 9.27963389341312])"
36,kenny53,捷絲旅礁溪館,"[客房, 空間, 傳統, 洋式, 東西, 晚餐, 文具店, 泡湯]",644,318,12.829105,0.490480,"(644, [644.0, 12.82910468155751])"
37,kenny53,捷絲旅,"[客房, 空間, 傳統, 洋式, 東西, 晚餐, 文具店, 泡湯]",644,315,12.829105,0.490480,"(644, [644.0, 12.82910468155751])"


In [104]:
# 对过滤之后的内容进行排序
df_use_for_prediction['rec_score'] = model.predict(user_ids=user_x, item_ids=df_use_for_prediction['hotel_int'].tolist(),
                                                   user_features=users_features)

df_use_for_prediction.head(3)

,author,hotel_name,keywords,uid_int,hotel_int,key_feature,uid_int_small,users_features,rec_score
2,terry910134,麗翔酒店,"[大廳, 泳池, 飯店, 鐵道, 客房, 南洋, 泡湯, 設施]",1120,789,12.531857,0.853008,"(1120, [1120.0, 12.531856834044447])",2.149405
21,sailly,和風時尚會館,"[房間, 飯店, 浴缸, 網路, 空間, 家庭, 備品, 宜蘭]",986,130,13.864929,0.750952,"(986, [986.0, 13.86492861044826])",3.432283
22,sailly,頭城時尚會館,"[房間, 飯店, 浴缸, 網路, 空間, 家庭, 備品, 宜蘭]",986,756,13.864929,0.750952,"(986, [986.0, 13.86492861044826])",1.247633


In [105]:
# 取 topk
top_rec_item = 5
rec_list_ = df_use_for_prediction.sort_values(by='rec_score', ascending=False)[:top_rec_item]['hotel_int']

# 排序后的电影推荐列表
rec_list_.tolist()

[32, 57, 53, 138, 27]

In [109]:
# 利用推荐列表找出电影信息
# result = df.loc[df['hotle_int'].isin(rec_list_.tolist())][[
#     'author', 'item_int', 'Movie_Score', 'Movie_Tags']].drop_duplicates('Movie_Name', keep='first', inplace=False)
result = df.loc[df['hotel_int'].isin(rec_list_.tolist())][[
    'author', 'hotel_name', 'keywords']].drop_duplicates('hotel_name', keep='first', inplace=False)
# 推荐结果显示
result.head(5)

,author,hotel_name,keywords
388,jlv1013,伯斯飯店,"[根本, 浪費, 房間, 飯店, 狀況, 早餐, 夜市, 親和力]"
552,sacy11230,單廊民宿,"[民宿, 單廊, 時間, 早餐, 簡潔, 中意, 硬度, 機會]"
2492,terry910134,丹迪旅店,"[旅店, 客房, 浴缸, 飯店, 天津, 商旅, 台北市, 早餐]"
4024,medea0926,信星旅館,"[旅館, 早餐, 台灣, 韓國, 原本, 房間, 連假, 衣服]"
4188,mikageyoru,九份愛琴海民宿,"[民宿, 早餐, 愛琴海, 咖啡, 房間, 用餐, 粉紅色, 客人]"


CSR_matrix 求餘旋相似度
相关用户推荐

 1.相关用户推荐，找到用户相关的用户，挖掘用户的潜在兴趣。 \
 2.增加平台和用户的交互。

In [111]:
#  提取用户特征向量
embedding_user, feature_user = csr_matrix(
    model.user_embeddings), csr_matrix(users_features)

# 用户特征提取
user_inner_max = np.dot(feature_user, embedding_user)

# 用户之间求得余弦相似度，容易因用户数量过大造成 MemoryError
user_cosine_sim = cosine_similarity(user_inner_max)

In [112]:
# 找到与查询电影最近余弦距离上的电影
def sim_user_recommendations(user_name, cosine_sim, topk):
    # 用户名转换唯一的 id
    int_id = user_dict[user_name]
    # 去除自身 id，建立倒排索引
    sim_scores = sorted(
        list(enumerate(cosine_sim[int_id])),
        key=lambda x: x[1], reverse=True)[1:topk+1]
    return sim_scores

In [113]:
# 直接使用 Username 作为当前用户名 假設
Username = 'terry910134'
df.loc[df['author'].isin([Username])].drop_duplicates(
    'uid_int', keep='first', inplace=False).head()

,author,hotel_name,keywords,uid_int,hotel_int,key_feature,uid_int_small,users_features
1,terry910134,麗翔,"[大廳, 泳池, 飯店, 鐵道, 客房, 南洋, 泡湯, 設施]",1120,788,12.531857,0.853008,"(1120, [1120.0, 12.531856834044447])"


In [128]:
topk = 10
sim_user_list = [i[0] for i in sim_user_recommendations(Username, user_cosine_sim, topk)]
sim_user_list

[32, 796, 17, 575, 889, 1066, 70, 1018, 60, 1114]

In [129]:
sim_user_result = df.loc[df['uid_int'].isin(sim_user_list)].drop_duplicates('hotel_name', keep='first', inplace=False)
# sim_user_result = df.loc[df['uid_int'].isin(sim_user_list)]

# todo：可以添加优质用户筛选策略
# Threshold 
# sim_user_result = sim_user_result[sim_user_result['User_level'] >= Threshold]

# 显示相关用户结果
sim_user_result.head()[['author','hotel_name']]

,author,hotel_name
66,shangyikuo,兆品
67,shangyikuo,兆品酒店
262,shangyikuo,水舍
299,shangyikuo,東旅
385,DoubleLiNg,樂亞香草藝術旅店


In [130]:
df.loc[df['uid_int']==889]

,author,hotel_name,keywords,uid_int,hotel_int,key_feature,uid_int_small,users_features
1224,onepercent,布諾斯莊園,"[民宿, 浴室, 主人, 落地窗, 感覺, 咖啡, 家庭, 夜市]",889,257,14.050489,0.677075,"(889, [889.0, 14.050488681561546])"


In [ ]:
# num_train = int(len(user_ids) * 0.8)
# train_user_ids = user_ids[:num_train]
# train_hotel_ids = hotel_ids[:num_train]
# train_features = df.key_feature.values[:num_train]

# val_user_ids = user_ids[num_train:]
# train_hotel_ids = hotel_ids[num_train:]
# val_ratings = df.key_feature[num_train:]

In [ ]:
# print(type(train_user_ids))
# print(val_ratings.shape)

In [ ]:
# # set up user-hotel matrix
# num_users = user_ids.max() + 1
# num_hotels = hotel_ids.max() + 1
# user2hotel = np.zeros([num_users, num_hotels])
# user2hotel[train_user_ids, train_hotel_ids] = train_features

# # user2hotel.shape

In [ ]:
def create_features(dataframe, features_name, id_col_name):
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features1 = list(zip(dataframe[id_col_name], features))
    features2 = features.apply(pd.Series).stack().reset_index(drop=True)
    return features1, features2

In [ ]:
# # 电影特征
# items_f = ['Movie_Score', 'Review_Count','item_int' ]

# 用户特征
users_f = ['uid_int','keywords']

# df['items_features'], item_feature_list = create_features(
#     df, items_f, 'item_int')

df['users_features'], user_feature_list = create_features(
    df, users_f, 'uid_int')


df.head(1)